In [2]:
import os
import itertools

In [3]:
def mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [4]:
# global job parameters

job_directory = f"math"
out_dir = f'{job_directory}/.out'
time_str = '00-12:00:00'
partition = 'gpu'
ntasks = 1
nodes = 1
cpu_per_task = 8
mem_per_cpu = 5
n_gpus = 1
# gpus_constraints = '"a100|rtx3090|v100|rtx2080ti"' # all gpus are pretty good now
project_dir = "/home/ma2393/project/abstract_transformer/experiments/math"

mkdir(job_directory)
mkdir(out_dir)

In [5]:
# define params of individual jobs

model_params = [
    dict(n_layers=1, d_model=128, n_heads=8, dff=128*2, symbol_type='symbolic_attention'),
    dict(n_layers=2, d_model=128, n_heads=8, dff=128*2, symbol_type='symbolic_attention')
]

In [6]:
tasks =  ['algebra__linear_1d', 'polynomials__add', 'polynomials__expand', 'calculus__differentiate', 'algebra__sequence_next_term']
n_epochs = 100
batch_size = 512

In [7]:
jobs_params = []
for model_p, task in itertools.product(model_params, tasks):
    jobs_params.append({**model_p, 'task': task, 'n_epochs': n_epochs, 'batch_size': batch_size})
print(jobs_params)
print(len(jobs_params))

[{'n_layers': 1, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'algebra__linear_1d', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 1, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'polynomials__add', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 1, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'polynomials__expand', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 1, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'calculus__differentiate', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 1, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'algebra__sequence_next_term', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 2, 'd_model': 128, 'n_heads': 8, 'dff': 256, 'symbol_type': 'symbolic_attention', 'task': 'algebra__linear_1d', 'n_epochs': 100, 'batch_size': 512}, {'n_layers': 2

In [8]:
# create jobs
created_jobs = []
for params in jobs_params:

    job_name = f"math-abstractor--{params['task']}-n_layers={params['n_layers']}"

    job_file = os.path.join(job_directory, f"{job_name}.job")

    with open(job_file, 'w') as fh:
        fh.writelines(f"#!/bin/bash\n")
        fh.writelines(f"#SBATCH --partition={partition}\n")
        fh.writelines(f"#SBATCH --job-name={job_name}\n")
        fh.writelines(f"#SBATCH --output={out_dir}/%j-{job_name}.out\n")
        fh.writelines(f"#SBATCH --ntasks={ntasks} --nodes={nodes}\n")
        fh.writelines(f"#SBATCH --cpus-per-task={cpu_per_task}\n")
        fh.writelines(f"#SBATCH --mem-per-cpu={mem_per_cpu}G\n")
        fh.writelines(f"#SBATCH --time={time_str}\n")
        fh.writelines(f"#SBATCH --mail-type=ALL\n")
        fh.writelines(f"#SBATCH --gpus={n_gpus}\n")
        # fh.writelines(f"#SBATCH -C {gpus_constraints}\n")# --gpus={n_gpus}\n")

        fh.writelines('\n')
        fh.writelines('module load StdEnv\n')
        fh.writelines('export SLURM_EXPORT_ENV=ALL\n')
        fh.writelines('\n')

        # fh.writelines(f"module restore python_env\n") # load modules i need
        fh.writelines(f"module load miniconda\n") # load modules i need
        # fh.writelines(f"conda init\n")
        fh.writelines(f"conda activate abstract_transformer\n") # activate conda environment
        fh.writelines(f"conda info --envs\n") # activate conda environment

        fh.writelines('\n')
        fh.writelines(f"nvidia-smi -L\n") # print gpu information
        fh.writelines('\n')

        fh.writelines(f"cd {project_dir}\n") # navigate to project directory
        # run python script
        fh.writelines(f"python train_abstractor_model.py ")
        fh.writelines(f"--task {params['task']} --n_epochs {params['n_epochs']} --batch_size {params['batch_size']} ")
        fh.writelines(f"--d_model {params['d_model']} --dff {params['dff']} --symbol_type {params['symbol_type']} ")
        fh.writelines(f"--n_layers {params['n_layers']} --n_heads {params['n_heads']} ")
        fh.writelines('\n')
    created_jobs.append(job_file)

In [9]:
n_trails = 5

In [10]:
proceed = input("Run all jobs? [Y/y or N/n]")
if proceed in ('Y', 'y'):
    for i in range(n_trails):
        for job in created_jobs:
            os.system(f'sbatch {job}')

Submitted batch job 25814
Submitted batch job 25815
Submitted batch job 25816
Submitted batch job 25817
Submitted batch job 25818
Submitted batch job 25819
Submitted batch job 25820
Submitted batch job 25821
Submitted batch job 25822
Submitted batch job 25823
Submitted batch job 25824
Submitted batch job 25825
Submitted batch job 25826
Submitted batch job 25827
Submitted batch job 25828
Submitted batch job 25829
Submitted batch job 25830
Submitted batch job 25831
Submitted batch job 25832
Submitted batch job 25833
Submitted batch job 25834
Submitted batch job 25835
Submitted batch job 25836
Submitted batch job 25837
Submitted batch job 25838
Submitted batch job 25839
Submitted batch job 25840
Submitted batch job 25841
Submitted batch job 25842
Submitted batch job 25843
Submitted batch job 25844
Submitted batch job 25845
Submitted batch job 25846
Submitted batch job 25847
Submitted batch job 25848
Submitted batch job 25849
Submitted batch job 25850
Submitted batch job 25851
Submitted ba

In [11]:
created_jobs

['math/math-abstractor--algebra__linear_1d-n_layers=1.job',
 'math/math-abstractor--polynomials__add-n_layers=1.job',
 'math/math-abstractor--polynomials__expand-n_layers=1.job',
 'math/math-abstractor--calculus__differentiate-n_layers=1.job',
 'math/math-abstractor--algebra__sequence_next_term-n_layers=1.job',
 'math/math-abstractor--algebra__linear_1d-n_layers=2.job',
 'math/math-abstractor--polynomials__add-n_layers=2.job',
 'math/math-abstractor--polynomials__expand-n_layers=2.job',
 'math/math-abstractor--calculus__differentiate-n_layers=2.job',
 'math/math-abstractor--algebra__sequence_next_term-n_layers=2.job']